In [170]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error
from scipy.special import expit  # Sigmoid function for propensity score
from kernel_regression import KernelRegression

In [172]:
# Simulation parameters
n = 5000 # Number of samples
p = 3    # Number of covariates
treatment_effect = 3.0  # True treatment effect

# Generate covariates
np.random.seed(0)

### Data generation

In [173]:
X = np.random.normal(0, 1, (n, p))

# Define a propensity score model
# Assume treatment probability is a sigmoid function of a subset of covariates
propensity_coef = np.random.normal(0, 0.5, p)
propensity_scores = expit(X @ propensity_coef)  # Calculate propensity scores

# Generate treatment assignment based on propensity scores
T = np.random.binomial(1, propensity_scores)

# Generate outcome with treatment effect
# Assume a simple linear model for demonstration
beta = np.random.normal(0, 1, p)
Y = (X @ beta)**2 + 1.1 + treatment_effect * T + np.random.normal(0, 1, n)

In [174]:
X_treatment = X[T == 1]
X_control = X[T == 0]

Y_treatment = Y[T == 1]
Y_control = Y[T == 0]

In [175]:
print(f"True Average Treatment Effect (ATE): {true_ATE}")

True Average Treatment Effect (ATE): 3.0


In [176]:
# Fit a linear model to estimate the treatment effect
model = LinearRegression()
model.fit(np.hstack([X, T.reshape(-1, 1)]), Y)
estimated_treatment_effect = model.coef_[-1]

# Evaluate performance
true_ATE = treatment_effect
bias = estimated_treatment_effect - true_ATE
mse = mean_squared_error(Y, model.predict(np.hstack([X, T.reshape(-1, 1)])))

print(f"Estimated ATE: {estimated_treatment_effect}")
print(f"Bias: {bias}")
print(f"Mean Squared Error: {mse}")

Estimated ATE: 2.972625870114331
Bias: -0.027374129885668896
Mean Squared Error: 1.7797254450419215


In [177]:
# Fit a linear model to estimate the treatment effect
prop_model = LogisticRegression()
prop_model.fit(X, T)
est_prop_score = prop_model.predict_proba(X)[:, 1]

treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

treatment_outcome_model.fit(X_treatment, Y_treatment)
control_outcome_model.fit(X_control, Y_control)

est_treatment_outcome = treatment_outcome_model.predict(X)
est_control_outcome = control_outcome_model.predict(X)

IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

# Evaluate performance
IPW_bias = IPW_est - true_ATE

print(f"Estimated ATE: {IPW_est}")
print(f"Bias: {IPW_bias}")

DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

# Evaluate performance
DR_bias = DR_est - true_ATE

print(f"Estimated ATE: {DR_est}")
print(f"Bias: {DR_bias}")

DM_est = np.mean(est_treatment_outcome - est_control_outcome)

# Evaluate performance
DM_bias = DM_est - true_ATE

print(f"Estimated ATE: {DM_est}")
print(f"Bias: {DM_bias}")

Estimated ATE: 2.887012893946858
Bias: -0.11298710605314222
Estimated ATE: 2.9749627896566637
Bias: -0.02503721034333628
Estimated ATE: 2.97911273489089
Bias: -0.020887265109109876


In [178]:
import numpy as np
from scipy.optimize import minimize

class DirectBiasCorrection:
    def __init__(self, initial_theta=None, max_iter=1000, tol=1e-6):
        """
        カスタムロジスティック回帰モデル
        :param initial_theta: 初期値 (デフォルトは None でゼロベクトルを使用)
        :param max_iter: 最大反復回数
        :param tol: 収束許容誤差
        """
        self.initial_theta = initial_theta
        self.max_iter = max_iter
        self.tol = tol
        self.theta_ = None  # 学習されたパラメータ

    def _sigmoid(self, X, theta):
        """シグモイド関数の定義"""
        return 1 / (1 + np.exp(-np.dot(X, theta)))

    def _loss_function(self, theta, X, T):
        """損失関数の定義"""
        outputs = self._sigmoid(X, theta)
        loss = -2 * (1 / outputs + 1 / (1 - outputs)) + (T / outputs - (1 - T) / (1 - outputs)) ** 2
        return np.mean(loss)

    def fit(self, X, T):
        """
        モデルの学習
        :param X: 説明変数（N×d の配列）
        :param T: 目的変数（N×1 のバイナリ配列）
        """
        X, T = np.asarray(X), np.asarray(T)

        # 初期値の設定（ゼロベクトル）
        if self.initial_theta is None:
            self.initial_theta = np.zeros(X.shape[1])

        # 最適化の実行
        result = minimize(
            self._loss_function, 
            self.initial_theta, 
            args=(X, T), 
            method='L-BFGS-B',
            options={'maxiter': self.max_iter, 'disp': False, 'ftol': self.tol}
        )
        
        self.theta_ = result.x

    def predict_proba(self, X):
        """
        予測確率の計算
        :param X: 説明変数（N×d の配列）
        :return: クラス1の確率
        """
        if self.theta_ is None:
            raise ValueError("Model is not fitted yet. Call 'fit' first.")
            
        output = self._sigmoid(X, self.theta_)
        return np.array([1-output, output]).T

    def predict(self, X):
        """
        クラスラベルの予測
        :param X: 説明変数（N×d の配列）
        :return: 0または1の予測クラス
        """
        probas = self.predict_proba(X)
        return (probas >= 0.5).astype(int)

    def get_params(self):
        """
        学習済みパラメータの取得
        :return: 推定された θ
        """
        if self.theta_ is None:
            raise ValueError("Model is not fitted yet. Call 'fit' first.")
        return self.theta_

In [179]:
# Fit a linear model to estimate the treatment effect
prop_model = DirectBiasCorrection()
prop_model.fit(X, T)
est_prop_score = prop_model.predict_proba(X)[:, 1]

treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

treatment_outcome_model.fit(X_treatment, Y_treatment)
control_outcome_model.fit(X_control, Y_control)

est_treatment_outcome = treatment_outcome_model.predict(X)
est_control_outcome = control_outcome_model.predict(X)

IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

# Evaluate performance
IPW_bias = IPW_est - true_ATE

print(f"Estimated ATE: {IPW_est}")
print(f"Bias: {IPW_bias}")

DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

# Evaluate performance
DR_bias = DR_est - true_ATE

print(f"Estimated ATE: {DR_est}")
print(f"Bias: {DR_bias}")

DM_est = np.mean(est_treatment_outcome - est_control_outcome)

# Evaluate performance
DM_bias = DM_est - true_ATE

print(f"Estimated ATE: {DM_est}")
print(f"Bias: {DM_bias}")

Estimated ATE: 2.9685072990904686
Bias: -0.03149270090953138
Estimated ATE: 2.972925625090657
Bias: -0.027074374909342946
Estimated ATE: 2.97911273489089
Bias: -0.020887265109109876


In [180]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
import pandas as pd
import numpy as np

# Enable automatic conversion of Pandas DataFrame to R DataFrame
pandas2ri.activate()

# Simulate data in Python

# Create a pandas DataFrame
column_names = [f'X{i+1}' for i in range(p)]
df = pd.DataFrame(X, columns=column_names)
df['T'] = T
df['Y'] = Y


# Convert pandas DataFrame to R DataFrame
r_df = pandas2ri.py2rpy(df)

ro.r.assign("p", p)

# Load the CBPS package in R and fit the model for ATE estimation
ro.r('''
    library(CBPS)
    estimate_cbps_ate <- function(df) {
        formula_str <- paste("T ~", paste(names(df)[1:{p}], collapse=" + "))
        
        # CBPSの適用 (ATEの推定、ATT=0)
        model <- CBPS(as.formula(formula_str), data = df, ATT = 0, method = "exact")
        
        # 推定された傾向スコアの取得
        df$propensity_score <- fitted(model)
        
        # IPW (Inverse Probability Weighting) を適用
        df$weight <- ifelse(df$T == 1, 1 / df$propensity_score, 1 / (1 - df$propensity_score))
        
        # 重み付き回帰によるATEの推定
        result <- lm(Y ~ T, data = df, weights = df$weight)
        
        return(df$propensity_score)
    }
''')

# R関数を呼び出してATEと傾向スコアを取得
est_prop_score = ro.r['estimate_cbps_ate'](r_df)

treatment_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))
control_outcome_model = KernelRegression(kernel="rbf", gamma=np.logspace(-2, 2, 10))

treatment_outcome_model.fit(X_treatment, Y_treatment)
control_outcome_model.fit(X_control, Y_control)

est_treatment_outcome = treatment_outcome_model.predict(X)
est_control_outcome = control_outcome_model.predict(X)

IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

# Evaluate performance
IPW_bias = IPW_est - true_ATE

print(f"Estimated ATE: {IPW_est}")
print(f"Bias: {IPW_bias}")

DR_est = np.mean(T*(Y - est_treatment_outcome) / est_prop_score - (1 - T)*(Y - est_control_outcome)  / (1 - est_prop_score) + est_treatment_outcome - est_control_outcome)

# Evaluate performance
DR_bias = DR_est - true_ATE

print(f"Estimated ATE: {DR_est}")
print(f"Bias: {DR_bias}")

DM_est = np.mean(est_treatment_outcome - est_control_outcome)

# Evaluate performance
DM_bias = DM_est - true_ATE

print(f"Estimated ATE: {DM_est}")
print(f"Bias: {DM_bias}")

Estimated ATE: 2.9577300335945154
Bias: -0.04226996640548464
Estimated ATE: 2.975807993897503
Bias: -0.024192006102496943
Estimated ATE: 2.97911273489089
Bias: -0.020887265109109876


In [144]:
est_prop_score

<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x326c98250> [3]
R classes: ('function',)

In [113]:
DM_est = np.mean(est_treatment_outcome - est_control_outcome)

# Evaluate performance
DM_bias = DM_est - true_ATE

print(f"Estimated ATE: {DM_est}")
print(f"Bias: {DM_bias}")

Estimated ATE: 2.9181776676318285
Bias: -0.08182233236817149


In [114]:


IPW_est = np.mean(T*Y / est_prop_score - (1 - T)*Y / (1 - est_prop_score))

# Evaluate performance
IPW_bias = IPW_est - true_ATE

print(f"Estimated ATE: {IPW_est}")
print(f"Bias: {IPW_bias}")

Estimated ATE: 3.067399653974786
Bias: 0.06739965397478587


In [115]:
T

array([1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,

In [116]:
import torch
import torch.nn as nn
import torch.optim as optim


KeyboardInterrupt



In [ ]:
# Convert data to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
T_tensor = torch.tensor(T, dtype=torch.float32).view(-1, 1)

# Define a simple neural network model for propensity score estimation
class PropensityScoreNN(nn.Module):
    def __init__(self, input_dim):
        super(PropensityScoreNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Initialize model, loss function, and optimizer
model = PropensityScoreNN(p)
criterion = nn.BCELoss()  # Binary Cross Entropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the neural network
num_epochs = 500
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(X_tensor)
    loss = criterion(outputs, T_tensor)
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    # Print loss for every 100 epochs
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Estimated propensity scores for all data
with torch.no_grad():
    estimated_propensity_scores = model(X_tensor).numpy()

In [187]:
estimated_propensity_scores[estimated_propensity_scores < 0.01] = 0.01

In [188]:
estimated_propensity_scores[estimated_propensity_scores > 0.99] = 0.99

In [181]:
np.mean((T/estimated_propensity_scores.T[0] - (1-T)/(1 - estimated_propensity_scores.T[0]))*Y)

2.011218015922772

In [194]:
# Convert data to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
T_tensor = torch.tensor(T, dtype=torch.float32).view(-1, 1)

# Define a simple neural network model for propensity score estimation
class PropensityScoreNN(nn.Module):
    def __init__(self, input_dim):
        super(PropensityScoreNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Initialize model, loss function, and optimizer
model = PropensityScoreNN(p)
criterion = nn.BCELoss()  # Binary Cross Entropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the neural network
num_epochs = 500
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(X_tensor)
    outputs = torch.clamp(outputs, min=0.01, max=0.99)
    loss = -2*(1/outputs + 1/(1 - outputs)) + (T_tensor / outputs - (1-T_tensor) / (1-outputs))**2
    loss = loss.mean()
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    # Print loss for every 100 epochs
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Estimated propensity scores for all data
with torch.no_grad():
    estimated_propensity_scores = model(X_tensor).numpy()

Epoch [100/500], Loss: -89.3757
Epoch [200/500], Loss: -77.3542
Epoch [300/500], Loss: -126.3577
Epoch [400/500], Loss: -137.7103
Epoch [500/500], Loss: -146.9600


In [189]:
Z_tensor = torch.cat([T_tensor, X_tensor], axis=1)
Y_tensor = torch.tensor(Y, dtype=torch.float32).view(-1, 1)
dim = Z_tensor.shape[1]

class CodOutcomeNN(nn.Module):
    def __init__(self, input_dim):
        super(CodOutcomeNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    
# Initialize model, loss function, and optimizer
model = CodOutcomeNN(dim)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Train the neural network
num_epochs = 10000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(Z_tensor)
    loss = ((Y_tensor - outputs)**2).mean()
    loss = loss.mean()
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    # Print loss for every 100 epochs
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Estimated propensity scores for all data
with torch.no_grad():
    estimated_conditional_outcomes = model(Z_tensor).numpy()
    estimated_conditional_outcomes_1 = model(Z_tensor_1).numpy()
    estimated_conditional_outcomes_0 = model(Z_tensor_0).numpy()

Epoch [100/10000], Loss: 66.5176
Epoch [200/10000], Loss: 55.3388
Epoch [300/10000], Loss: 43.7221
Epoch [400/10000], Loss: 34.2139
Epoch [500/10000], Loss: 28.5578
Epoch [600/10000], Loss: 25.6987
Epoch [700/10000], Loss: 23.8395
Epoch [800/10000], Loss: 22.1396
Epoch [900/10000], Loss: 20.4078
Epoch [1000/10000], Loss: 18.6166
Epoch [1100/10000], Loss: 16.7452
Epoch [1200/10000], Loss: 14.7996
Epoch [1300/10000], Loss: 12.8643
Epoch [1400/10000], Loss: 10.9603
Epoch [1500/10000], Loss: 9.1128
Epoch [1600/10000], Loss: 7.3873
Epoch [1700/10000], Loss: 5.9171
Epoch [1800/10000], Loss: 4.7640
Epoch [1900/10000], Loss: 3.9100
Epoch [2000/10000], Loss: 3.3287
Epoch [2100/10000], Loss: 2.9629
Epoch [2200/10000], Loss: 2.7293
Epoch [2300/10000], Loss: 2.5687
Epoch [2400/10000], Loss: 2.4482
Epoch [2500/10000], Loss: 2.3496
Epoch [2600/10000], Loss: 2.2630
Epoch [2700/10000], Loss: 2.1825
Epoch [2800/10000], Loss: 2.1069
Epoch [2900/10000], Loss: 2.0346
Epoch [3000/10000], Loss: 1.9642
Epoch

In [190]:
with torch.no_grad():
    estimated_conditional_outcomes = model(Z_tensor).numpy()
    estimated_conditional_outcomes_1 = model(Z_tensor_1).numpy()
    estimated_conditional_outcomes_0 = model(Z_tensor_0).numpy()

In [191]:
aaa = np.mean(estimated_conditional_outcomes_1 - estimated_conditional_outcomes_0)

In [192]:
aaa

2.98163

In [193]:
aaa + np.mean((T/estimated_propensity_scores.T[0] - (1-T)/(1 - estimated_propensity_scores.T[0]))*(Y - estimated_conditional_outcomes.T[0]))

2.979416648378537

In [160]:
Y - estimated_conditional_outcomes.T[0]

array([ 5.63514307e-01,  4.77596318e-01,  6.29739734e-02, -1.27126940e-01,
       -1.77228238e-01,  4.64099619e-01, -3.51511405e-01, -2.99983419e-01,
       -2.69382789e-01,  1.26012672e-01, -3.40216497e-03,  2.07889849e-01,
       -6.60575762e-01, -4.97077542e-01, -8.41113774e-01, -1.26968687e-01,
       -4.25912768e-01, -9.33943451e-02, -4.43000561e-01,  2.44456645e-01,
        7.93688016e-02, -7.86699426e-01,  6.66016525e-01,  5.07777333e-01,
       -6.60204782e-01,  1.76161963e-01,  3.64499704e-01, -2.11489539e-01,
       -3.55552370e-01, -6.61527685e-01, -7.72481761e-02, -2.88697698e-01,
       -6.51645266e-01, -1.14941385e+00, -5.81560010e-01,  9.66436085e-01,
       -7.85939945e-01, -4.66742301e-01,  7.49621729e-01,  2.88066177e-01,
       -5.62568961e-01, -1.75046911e-01,  2.87174176e-01,  5.13794861e-01,
       -3.57863637e-01, -6.20851911e-01, -4.15798049e-01, -4.12468909e-01,
        2.46813750e-01, -5.61748112e-01,  7.15916546e-01, -6.73407462e-01,
       -1.76732254e-01, -